# Platooning algorithms comparison

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
import time
from tqdm import tqdm

In [ ]:
from dyn.linear_long import LinearLong
from dyn.nonlinear_long import NonlinearLong
from alg.linear_feedback import LinearFeedback
from alg.linear_mpc import LinearMPC
from alg.platoon_linear_mpc import PlatoonLinearMPC

## Platoon linear feedback

Recreating results (roughly) from Zheng et al. "Stability and scalability of 
homogeneous vehicular platoon: Study on the influence of information flow 
topologies."

In [ ]:
# dynamics setup
dt = 0.1
x_min = np.array([-np.inf, -np.inf, -np.inf])
x_max = np.array([np.inf, np.inf, np.inf])
u_min = np.array([-5.0])
u_max = np.array([5.0])
tau = 0.5
linear_dyn = LinearLong(dt, x_min, x_max, u_min, u_max, tau)

In [ ]:
# controller setup
k_p = 1
k_v = 2
k_a = 1
lin_fbk = LinearFeedback(k_p, k_v, k_a)

### The case of unbounded inputs

In [ ]:
# simulation setup
N = 11  # 1 leader, 10 followers
N_fbk = N
t_range = np.arange(start=0.0, stop=40.+dt, step=dt)
T = len(t_range)

s_unbounded = [np.zeros(T) for _ in range(N + 1)]  # positions
v_unbounded = [np.zeros(T) for _ in range(N + 1)]  # velocities
a_unbounded = [np.zeros(T) for _ in range(N + 1)]  # accelerations
d_unbounded = [np.zeros(T) for _ in range(N + 1)]  # distances
u_unbounded = [np.zeros(T - 1) for _ in range(N + 1)]  # input torques

# leader reference
mask = np.logical_and(t_range > 5., t_range <= 10.)
v_ref = np.zeros(T)
v_ref[t_range <= 5.] = 20.
v_ref[mask] = 20 + 2 * (t_range[mask] - t_range[mask][0])
v_ref[t_range > 10.] = 30.

a_ref = np.zeros(T)
a_ref[mask] = 2.

s_ref = np.zeros(T)
s_ref[t_range <= 5.] = 20 * t_range[t_range <= 5.]
s_ref[mask] = (t_range[mask] - 5)**2 + 20 * (t_range[mask] - 5) + s_ref[t_range <= 5.][-1]
s_ref[t_range > 10.] = 30 * (t_range[t_range > 10.] - 10.) + s_ref[mask][-1]

x_ref = np.r_[s_ref, v_ref, a_ref].reshape((3, -1))

d_des = 20.  # desired distance

# initial values for platoon
for i in range(N + 1):
    s_unbounded[i][0] = -i * d_des
    v_unbounded[i][0] = v_ref[0]
    a_unbounded[i][0] = 0.

In [ ]:
# leader dynamics 
for k in range(T - 1):
    x = np.array([s_unbounded[0][k], v_unbounded[0][k], a_unbounded[0][k]])  # state

    # calculate control
    pos_err = s_unbounded[0][k] - s_ref[k]
    vel_err = v_unbounded[0][k] - v_ref[k]
    accel_err =  a_unbounded[0][k] - a_ref[k]
    d_unbounded[0][k] = pos_err
    u = lin_fbk.control(pos_err, vel_err, accel_err)
    u_unbounded[0][k] = u

    # get next state
    x_new = linear_dyn.forward(x, np.array([u]))
    s_unbounded[0][k+1] = x_new[0]
    v_unbounded[0][k+1] = x_new[1]
    a_unbounded[0][k+1] = x_new[2]

In [ ]:
# platoon dynamics (predecessor follower)
for k in range(T - 1):
    for i in range(1, N + 1):
        x = np.array([s_unbounded[i][k], v_unbounded[i][k], a_unbounded[i][k]])

        # calculate control (predecessor follower topology)
        pos_err = d_des - (s_unbounded[i-1][k] - s_unbounded[i][k])
        vel_err = v_unbounded[i][k] - v_unbounded[i-1][k]
        accel_err = a_unbounded[i][k] - a_unbounded[i-1][k]
        d_unbounded[i][k] = pos_err + d_des
        u = lin_fbk.control(pos_err, vel_err, accel_err)
        u_unbounded[i][k] = u

        # get next state
        x_new = linear_dyn.forward(x, np.array([u]))
        s_unbounded[i][k+1] = x_new[0]
        v_unbounded[i][k+1] = x_new[1]
        a_unbounded[i][k+1] = x_new[2]

for i in range(1, N + 1):
    d_unbounded[i][T-1] = s_unbounded[i-1][T-1] - s_unbounded[i][T-1]

In [ ]:
# save data for later use
x_fbk_unbounded = [np.zeros((linear_dyn.n, T)) for _ in range(N)]
for n in range(N):
    for t in range(T):
        x_fbk_unbounded[n][0, t] = s_unbounded[n][t]
        x_fbk_unbounded[n][1, t] = v_unbounded[n][t]
        x_fbk_unbounded[n][2, t] = a_unbounded[n][t]

### The case of bounded inputs

In [ ]:
# simulation setup
N = 11  # 1 leader, 10 followers
t_range = np.arange(start=0.0, stop=40.01, step=dt)
T = len(t_range)

s_bounded = [np.zeros(T) for _ in range(N + 1)]  # positions
v_bounded = [np.zeros(T) for _ in range(N + 1)]  # velocities
a_bounded = [np.zeros(T) for _ in range(N + 1)]  # accelerations
d_bounded = [np.zeros(T) for _ in range(N + 1)]  # distances
u_bounded = [np.zeros(T - 1) for _ in range(N + 1)]  # input torques

# leader reference
mask = np.logical_and(t_range > 5., t_range <= 10.)
v_ref = np.zeros(T)
v_ref[t_range <= 5.] = 20.
v_ref[mask] = 20 + 2 * (t_range[mask] - t_range[mask][0])
v_ref[t_range > 10.] = 30.

a_ref = np.zeros(T)
a_ref[mask] = 2.

s_ref = np.zeros(T)
s_ref[t_range <= 5.] = 20 * t_range[t_range <= 5.]
s_ref[mask] = (t_range[mask] - 5)**2 + 20 * (t_range[mask] - 5) + s_ref[t_range <= 5.][-1]
s_ref[t_range > 10.] = 30 * (t_range[t_range > 10.] - 10.) + s_ref[mask][-1]

d_des = 20.  # desired distance

# initial values for platoon
for i in range(N + 1):
    s_bounded[i][0] = -i * d_des
    v_bounded[i][0] = v_ref[0]
    a_bounded[i][0] = 0.

In [ ]:
# leader dynamics 
for k in range(T - 1):
    x = np.array([s_bounded[0][k], v_bounded[0][k], a_bounded[0][k]])  # state

    # calculate control
    pos_err = s_bounded[0][k] - s_ref[k]
    vel_err = v_bounded[0][k] - v_ref[k]
    accel_err =  a_bounded[0][k] - a_ref[k]
    d_bounded[0][k] = pos_err
    u = lin_fbk.control(pos_err, vel_err, accel_err)
    u = np.clip(u, u_min, u_max)
    u_bounded[0][k] = u

    # get next state
    x_new = linear_dyn.forward(x, u)
    s_bounded[0][k+1] = x_new[0]
    v_bounded[0][k+1] = x_new[1]
    a_bounded[0][k+1] = x_new[2]

In [ ]:
# platoon dynamics (predecessor follower)
for k in range(T - 1):
    for i in range(1, N + 1):
        x = np.array([s_bounded[i][k], v_bounded[i][k], a_bounded[i][k]])

        # calculate control (predecessor follower topology)
        pos_err = d_des - (s_bounded[i-1][k] - s_bounded[i][k])
        vel_err = v_bounded[i][k] - v_bounded[i-1][k]
        accel_err = a_bounded[i][k] - a_bounded[i-1][k]
        d_bounded[i][k] = pos_err + d_des
        u = lin_fbk.control(pos_err, vel_err, accel_err)
        u = np.clip(u, u_min, u_max)
        u_bounded[i][k] = u

        # get next state
        x_new = linear_dyn.forward(x, u)
        s_bounded[i][k+1] = x_new[0]
        v_bounded[i][k+1] = x_new[1]
        a_bounded[i][k+1] = x_new[2]

for i in range(1, N + 1):
    d_bounded[i][T-1] = s_bounded[i-1][T-1] - s_bounded[i][T-1]

In [ ]:
# save data for later use
x_fbk_bounded = [np.zeros((linear_dyn.n, T)) for n in range(N)]
for n in range(N):
    for t in range(T):
        x_fbk_bounded[n][0, t] = s_bounded[n][t]
        x_fbk_bounded[n][1, t] = v_bounded[n][t]
        x_fbk_bounded[n][2, t] = a_bounded[n][t]

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(8, 8), sharex='col', sharey='row')
fig.suptitle("Platoon dynamics", size=16)
fig.subplots_adjust(top=0.92)
ax[0][0].set_title("Unbounded input")
ax[0][1].set_title("Bounded input")

ax[0][0].plot(t_range, d_des * np.ones(T), label=r"$d_{des}$")
ax[0][1].plot(t_range, d_des * np.ones(T), label=r"$d_{des}$")
ax[0][0].plot(t_range, np.zeros(T), label="crash")
ax[0][1].plot(t_range, np.zeros(T), label="crash")
for i in [1, 3, 5, 7, 8, 9, 10]:
    ax[0][0].plot(t_range, d_unbounded[i], label=f"{i}")
    ax[0][1].plot(t_range, d_bounded[i], label=f"{i}")

for i in range(0, N + 1, 2):
    ax[1][0].plot(t_range, v_unbounded[i], label=f"{i}")
    ax[1][1].plot(t_range, v_bounded[i], label=f"{i}")
    ax[2][0].plot(t_range, a_unbounded[i], label=f"{i}")
    ax[2][1].plot(t_range, a_bounded[i], label=f"{i}")
    ax[3][0].plot(t_range[:-1], u_unbounded[i], label=f"{i}")
    ax[3][1].plot(t_range[:-1], u_bounded[i], label=f"{i}")

ax[0][0].set_ylabel("distance [m]")
ax[0][0].grid()
ax[1][0].set_ylabel("velocity [m/s]")
ax[1][0].grid()
ax[2][0].set_ylabel(r"acceleration [m/s$^2$]")
ax[2][0].grid()
ax[3][0].set_xlabel("time [s]")
ax[3][0].set_ylabel(r"input acceleration [m/s$^2$]")
ax[3][0].grid()

ax[0][1].grid()
ax[0][1].legend(bbox_to_anchor=(1.02, 0.5), loc="center left")
ax[1][1].grid()
ax[1][1].legend(bbox_to_anchor=(1.02, 0.5), loc="center left")
ax[2][1].grid()
ax[2][1].legend(bbox_to_anchor=(1.02, 0.5), loc="center left")
ax[3][1].set_xlabel("time [s]")
ax[3][1].grid()
ax[3][1].legend(bbox_to_anchor=(1.02, 0.5), loc="center left");

As we can see, it crashes when there is a bounded input. This is bad. Also, the 
velocity and acceleration max magnitudes grow quickly with platoon size in both 
cases. This is also bad. And this is with a relatively simple reference leader 
trajectory (accelerating 10 m/s or approximately 22 mph in 5 seconds).

## Distributed MPC

Recreating results (roughly) from Zheng et al. "Distributed model predictive 
control for heterogeneous vehicle platoons under unidirectional topologies."

Before re-creating the paper results, we'll perform distributed MPC with the 
linearized longitudinal model to compare results to the linear feedback paper 
above.

In [ ]:
# dynamics setup (assume homogeneous platoon)
dt = 0.1
x_min = np.array([-np.inf, -np.inf, -np.inf])
x_max = np.array([np.inf, np.inf, np.inf])
u_min = np.array([-6.0])
u_max = np.array([6.0])
tau = 0.5
dyn = LinearLong(dt, x_min, x_max, u_min, u_max, tau)

In [ ]:
# controller setup

# leader MPC controller for tracking ref
Q = 10 * np.eye(dyn.n)
Q_f = 0.0 * np.eye(dyn.n)
R = 1.0 * np.eye(dyn.m)
H = 20
linear_mpc = LinearMPC(Q, Q_f, R, dyn, x_min, x_max, u_min, u_max, H, 
                       solver='cvx')

# follower MPC controller
Q = 10 * np.eye(dyn.p)
R = np.eye(dyn.m)
F = np.eye(dyn.p)
F[0, 0] = 10.
F[1, 1] = 2.
G = np.eye(dyn.p)
G[0, 0] = 2.
G[1, 1] = 2.
H = 20
platoon_mpc = PlatoonLinearMPC(Q, R, F, G, dyn, x_min, x_max, u_min, u_max, H) 

In [ ]:
# study two cases: predecessor-follower and bidirectional follower
N = 11

x_pfmpc_platoon = [np.zeros_like(x_ref) for _ in range(N + 1)]
u_pfmpc_platoon = [np.zeros((dyn.m, T - 1)) for _ in range(N + 1)]

x_bdmpc_platoon = [np.zeros_like(x_ref) for _ in range(N + 1)]
u_bdmpc_platoon = [np.zeros((dyn.m, T - 1)) for _ in range(N + 1)]

# initial condition
for i in range(N + 1):
    x_pfmpc_platoon[i][:, 0] = np.array([-i * d_des, x_ref[1, 0], x_ref[2, 0]])
    x_bdmpc_platoon[i][:, 0] = np.array([-i * d_des, x_ref[1, 0], x_ref[2, 0]])

# initialize assumed states, controls, and output for first timestep
x_a_pfmpc = [np.zeros((dyn.n, H + 1)) for _ in range(N + 1)]
x_a_bdmpc = [np.zeros((dyn.n, H + 1)) for _ in range(N + 1)]
for i in range(N + 1):
    x_a_pfmpc[i][:, 0] = x_pfmpc_platoon[i][:, 0]
    x_a_bdmpc[i][:, 0] = x_bdmpc_platoon[i][:, 0]

y_a_pfmpc = [np.zeros((dyn.p, H + 1)) for _ in range(N + 1)]
y_a_bdmpc = [np.zeros((dyn.p, H + 1)) for _ in range(N + 1)]
for i in range(N + 1):
    y_a_pfmpc[i][:, 0] = dyn.sense(x_a_pfmpc[i][:, 0])
    y_a_bdmpc[i][:, 0] = dyn.sense(x_a_bdmpc[i][:, 0])

u_a_pfmpc = [np.zeros((dyn.m, H)) for _ in range(N + 1)]  # zero accel
u_a_bdmpc = [np.zeros((dyn.m, H)) for _ in range(N + 1)]  # zero accel

for i in range(N + 1):
    for t in range(H):
        x_a_pfmpc[i][:, t+1] = dyn.forward(x_a_pfmpc[i][:, t], 
                                           u_a_pfmpc[i][:, t])
        y_a_pfmpc[i][:, t+1] = dyn.sense(x_a_pfmpc[i][:, t+1])

        x_a_bdmpc[i][:, t+1] = dyn.forward(x_a_bdmpc[i][:, t], 
                                           u_a_bdmpc[i][:, t])
        y_a_bdmpc[i][:, t+1] = dyn.sense(x_a_bdmpc[i][:, t+1])

x_a_pfmpc_prev = copy.deepcopy(x_a_pfmpc)
y_a_pfmpc_prev = copy.deepcopy(y_a_pfmpc)
u_a_pfmpc_prev = copy.deepcopy(u_a_pfmpc)

x_a_bdmpc_prev = copy.deepcopy(x_a_bdmpc)
y_a_bdmpc_prev = copy.deepcopy(y_a_bdmpc)
u_a_bdmpc_prev = copy.deepcopy(u_a_bdmpc)

In [ ]:
# predecessor follower case
start_time = time.time()
for t in tqdm(range(T - H)):
    for i in range(N + 1):

        # leader MPC tracking
        if i == 0:
            x_0 = x_pfmpc_platoon[i][:, t]
            z = x_ref[:, t:t+H+1]
            x_T = z[:, -1]
            u_opt, x_opt, value = linear_mpc.control(x_0, x_T, z)
            u = u_opt[:, 0]
            u_pfmpc_platoon[i][:, t] = u
            x_pfmpc_platoon[i][:, t+1] = dyn.forward(x_0, u)

        # follower MPC tracking
        else:
            x_0 = x_pfmpc_platoon[i][:, t]
            y_a = y_a_pfmpc_prev[i]
            y_neighbor_a = [y_a_pfmpc_prev[i-1]]
            d_tilde = [d_des]
            u_opt, x_opt, y_opt, value = platoon_mpc.control(x_0, y_a, 
                                                             y_neighbor_a, 
                                                             d_tilde)
            u = u_opt[:, 0]
            u_pfmpc_platoon[i][:, t] = u
            x_pfmpc_platoon[i][:, t+1] = dyn.forward(x_0, u)

        # assumed state, control, and output for next timestep
        u_a_pfmpc[i][:, :H-1] = u_opt[:, 1:H]
        u_a_pfmpc[i][:, H-1] = np.zeros((dyn.m))
        x_a_pfmpc[i][:, 0] = x_opt[:, 1]
        y_a_pfmpc[i][:, 0] = dyn.sense(x_a_pfmpc[i][:, 0])
        for k in range(H):
            x_a_pfmpc[i][:, k+1] = dyn.forward(x_a_pfmpc[i][:, k],
                                               u_a_pfmpc[i][:, k])
            y_a_pfmpc[i][:, k+1] = dyn.sense(x_a_pfmpc[i][:, k+1])

    x_a_pfmpc_prev = copy.deepcopy(x_a_pfmpc)
    y_a_pfmpc_prev = copy.deepcopy(y_a_pfmpc)
    u_a_pfmpc_prev = copy.deepcopy(u_a_pfmpc)

end_time = time.time()
pf_avg_time = (end_time - start_time) / ((N + 1) * (T - H))
print(f"pf-mpc average time: {pf_avg_time} s")

In [ ]:
# calculate spacing error
d_error_pfmpc = [np.zeros(T) for _ in range(N+1)]
v_error_pfmpc = [np.zeros(T) for _ in range(N+1)]
d_error_pfmpc[0] = x_pfmpc_platoon[0][0, :] - x_ref[0, :]
v_error_pfmpc[0] = x_pfmpc_platoon[0][1, :] - x_ref[1, :]
for i in range(1, N + 1):
    d_error_pfmpc[i] = x_pfmpc_platoon[i-1][0, :] - x_pfmpc_platoon[i][0, :] - d_des
    v_error_pfmpc[i] = x_pfmpc_platoon[i][1, :] - x_pfmpc_platoon[i-1][1, :]

In [ ]:
# bidirectional case
start_time = time.time()
for t in range(T - H):
    for i in range(N + 1):

        # leader MPC tracking
        if i == 0:
            x_0 = x_bdmpc_platoon[i][:, t]
            y_a = y_a_bdmpc_prev[i]
            y_neighbor_a = [x_ref[:2, t:t+H+1], y_a_bdmpc_prev[i+1]]
            d_tilde = [0.0, -d_des]
            u_opt, x_opt, y_opt, value = platoon_mpc.control(x_0, y_a, 
                                                             y_neighbor_a,
                                                             d_tilde)
            u = u_opt[:, 0]
            u_bdmpc_platoon[i][:, t] = u
            x_bdmpc_platoon[i][:, t+1] = dyn.forward(x_0, u)

        # follower MPC tracking
        else:
            x_0 = x_bdmpc_platoon[i][:, t]
            y_a = y_a_bdmpc_prev[i]
            if i < N:
                y_neighbor_a = [y_a_bdmpc_prev[i-1], y_a_bdmpc_prev[i+1]]
                d_tilde = [d_des, -d_des]
            else:
                y_neighbor_a = [y_a_bdmpc_prev[i-1]]
                d_tilde = [d_des]
            u_opt, x_opt, y_opt, value = platoon_mpc.control(x_0, y_a, 
                                                             y_neighbor_a, 
                                                             d_tilde)
            u = u_opt[:, 0]
            u_bdmpc_platoon[i][:, t] = u
            x_bdmpc_platoon[i][:, t+1] = dyn.forward(x_0, u)

        u_a_bdmpc[i][:, :H-1] = u_opt[:, 1:H]
        u_a_bdmpc[i][:, H-1] = np.zeros((dyn.m))
        x_a_bdmpc[i][:, 0] = x_opt[:, 1]
        y_a_bdmpc[i][:, 0] = dyn.sense(x_a_bdmpc[i][:, 0])
        for k in range(H):
            x_a_bdmpc[i][:, k+1] = dyn.forward(x_a_bdmpc[i][:, k],
                                               u_a_bdmpc[i][:, k])
            y_a_bdmpc[i][:, k+1] = dyn.sense(x_a_bdmpc[i][:, k+1])

    x_a_bdmpc_prev = copy.deepcopy(x_a_bdmpc)
    y_a_bdmpc_prev = copy.deepcopy(y_a_bdmpc)
    u_a_bdmpc_prev = copy.deepcopy(u_a_bdmpc)

end_time = time.time()
bd_avg_time = (end_time - start_time) / ((N + 1) * (T - H))
print(f"bd-mpc average time: {bd_avg_time} s")

In [ ]:
# calculate spacing error
d_error_bdmpc = [np.zeros(T) for _ in range(N+1)]
v_error_bdmpc = [np.zeros(T) for _ in range(N+1)]
d_error_bdmpc[0] = x_ref[0, :] - x_bdmpc_platoon[0][0, :]
v_error_bdmpc[0] = x_bdmpc_platoon[0][1, :]- x_ref[1, :]
for i in range(1, N + 1):
    d_error_bdmpc[i] = x_bdmpc_platoon[i-1][0, :] - x_bdmpc_platoon[i][0, :] - d_des
    v_error_bdmpc[i] = x_bdmpc_platoon[i][1, :] - x_bdmpc_platoon[i-1][1, :]

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(8, 8), sharex=True, sharey='row')
fig.suptitle("Platoon tracking dynamics: PF-MPC vs BD-MPC", size=16)
fig.subplots_adjust(top=0.92)

ax[0][0].set_title("PF-MPC")
ax[0][1].set_title("BD-MPC")

for i in range(1, N+1, 2):
    ax[0][0].plot(t_range[:T-H], d_error_pfmpc[i][:T-H], label=f"vehicle {i}")
    ax[0][1].plot(t_range[:T-H], d_error_bdmpc[i][:T-H], label=f"vehicle {i}")
ax[0][0].set_ylabel("spacing error [m]")
ax[0][0].grid()
ax[0][1].grid()

for i in range(1, N+1, 2):
    ax[1][0].plot(t_range[:T-H], v_error_pfmpc[i][:T-H], label=f"vehicle {i}")
    ax[1][1].plot(t_range[:T-H], v_error_bdmpc[i][:T-H], label=f"vehicle {i}")
ax[1][0].set_ylabel("velocity error [m/s]")
ax[1][0].grid()
ax[1][1].grid()
ax[1][1].legend(bbox_to_anchor=(1.02, 1.1), loc="center left")

ax[2][0].plot(t_range, x_ref[2, :], label="leader reference")
ax[2][1].plot(t_range, x_ref[2, :], label="leader reference")
for i in range(1, N+1, 2):
    ax[2][0].plot(t_range[:T-H], x_pfmpc_platoon[i][2, :T-H], label=f"vehicle {i}")
    ax[2][1].plot(t_range[:T-H], x_bdmpc_platoon[i][2, :T-H], label=f"vehicle {i}")
ax[2][0].set_ylabel(r"acceleration [m/s$^2$]")
ax[2][0].grid()
ax[2][1].grid()

ax[3][0].plot(t_range, x_ref[2, :], label="leader reference")
ax[3][1].plot(t_range, x_ref[2, :], label="leader reference")
for i in range(1, N+1, 2):
    ax[3][0].plot(t_range[:T-H], u_pfmpc_platoon[i][0, :T-H], label=f"vehicle {i}")
    ax[3][1].plot(t_range[:T-H], u_bdmpc_platoon[i][0, :T-H], label=f"vehicle {i}")
ax[3][0].set_xlabel("time [s]")
ax[3][1].set_xlabel("time [s]")
ax[3][0].set_ylabel(r"input acceleration [m/s$^2$]")
ax[3][0].grid()
ax[3][1].grid()
ax[3][1].legend(bbox_to_anchor=(1.02, 1.1), loc="center left");

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 4), sharey=True, sharex=True)
fig.suptitle("Spacing error: PF-MPC vs linear feedback", size=16)
fig.subplots_adjust(top=0.85)

ax[0].set_title("PF-MPC")
ax[1].set_title("Linear feedback - unbounded")
ax[2].set_title("Linear feedback - bounded")

for i in range(1, N_fbk):
    ax[0].plot(t_range[:T-H], d_error_pfmpc[i][:T-H], label=f"vehicle {i}")
    ax[1].plot(t_range, d_unbounded[i] - d_des, label=f"vehicle {i}")
    ax[2].plot(t_range, d_bounded[i] - d_des, label=f"vehicle {i}")
ax[0].set_xlabel("time [s]")
ax[1].set_xlabel("time [s]")
ax[2].set_xlabel("time [s]")
ax[0].set_ylabel("spacing error [m]")
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[2].legend(bbox_to_anchor=(1.02, 0.5), loc="center left");

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
fig.suptitle("Spacing error: PF-MPC vs BD-MPC", size=16)
fig.subplots_adjust(top=0.85)

ax[0].set_title("PF-MPC")
ax[1].set_title("BD-MPC")

for i in range(N):
    ax[0].plot(t_range[:T-H], d_error_pfmpc[i][:T-H], label=f"vehicle {i}")
    ax[1].plot(t_range[:T-H], d_error_bdmpc[i][:T-H], label=f"vehicle {i}")

ax[0].set_xlabel("time [s]")
ax[1].set_xlabel("time [s]")
ax[0].set_ylabel("spacing error [m]")
ax[0].grid()
ax[1].grid()
ax[1].legend(bbox_to_anchor=(1.02, 0.5), loc="center left");